In [ ]:
import urllib.request
import random
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = urllib.request.urlopen('https://raw.githubusercontent.com/karpathy/makemore/master/names.txt').read().decode('utf-8').splitlines()

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)
vocab_size = len(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [ ]:
# make dataset
block_size = 3
def build_dataset(words):
  X, Y = [], []

  for word in words:
    context = [0] * block_size
    word += '.'
    for ch in word:
      X.append(context)
      idx = stoi[ch]
      Y.append(idx)
      context = context[1:] + [idx]

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
train_words = words[:n1]
dev_words = words[n1:n2]
test_words = words[n2:]
Xtrain, Ytrain  = build_dataset(train_words)
Xdev,   Ydev    = build_dataset(dev_words)
Xtest,  Ytest   = build_dataset(test_words)

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
n_emb = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn(vocab_size, n_emb,                generator=g)
W1 = torch.randn((n_emb * block_size, n_hidden),   generator=g) * (5/3)/((n_emb * block_size)**0.5)
b1 = torch.randn((n_hidden,),                      generator=g) * 0.1
W2 = torch.randn((n_hidden, vocab_size),           generator=g) * 0.1
b2 = torch.randn((vocab_size),                      generator=g) * 0.1

bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]

p_size = sum(p.nelement() for p in parameters)
print(f"{p_size=}")
for p in parameters:
  p.requires_grad = True # if multiplying by constant, use requires_grad separately after



p_size=4137


In [ ]:
for i in range(1):
    print(i)

0


In [ ]:
max_steps = 1
batch_size = 32
lossi = []
stepi = []

for i in range(max_steps):
  #mini batch
  ix = torch.randint(0, Xtrain.shape[0], (batch_size,), generator=g)
  X_batch = Xtrain[ix]
  Y_batch = Ytrain[ix]
  ##forward
  emb = C[X_batch]
  embcat = emb.reshape(emb.shape[0], -1)

  # linear
  hprebn = embcat @ W1 + b1

  # batchnorm
  bnmeani = 1/batch_size * hprebn.sum(0, keepdims=True)
  bndiff = hprebn - bnmeani
  bndiff2 = bndiff**2
  bnvar = (1/(batch_size-1) * bndiff2).sum(0, keepdims=True)
  bnvar_inv = (bnvar + 1e-5)**-0.5
  bnraw = bndiff * bnvar_inv
  hpreact = bngain * bnraw + bnbias

  #non-linearity
  h = torch.tanh(hpreact)

  #linear layer 2
  logits = h @ W2 + b2
  print(logits.shape)
  #cross-entropy --> replacement of ==> loss = F.cross_entropy(logits, Y_batch)
  logit_maxes = logits.max(1, keepdim=True).values
  norm_logits = logits - logit_maxes # for numerical stability. else higher values's exp will become inf
  counts = norm_logits.exp()
  counts_sum = counts.sum(1, keepdim=True)
  counts_sum_inv = counts_sum**-1
  probs = counts * counts_sum_inv
  logprobs = probs.log()
  loss = -logprobs[torch.arange(batch_size), Y_batch].mean()


  # backward
  for p in parameters:
    p.grad = None
  for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()
  loss.backward()


  '''
  # grad descent
  #lr = 0.1 if i < max_steps*0.5 else 0.01
  #lr = (max_steps-i)/max_steps*0.1
  lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad
  # track
  lossi.append(loss.log10().item())
  stepi.append(i)

  if i%10000 == 0:
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  '''
print(loss.item())

torch.Size([32, 27])
3.354541063308716


In [ ]:
a11 a12 a13
a21 a22 a23
a31 a32 a33

SyntaxError: invalid syntax (<ipython-input-81-dca1cea2a96d>, line 1)

In [ ]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [ ]:
h.shape, W2.shape, b2.shape, logits.shape

#norm_logits = logits - logit_maxes
#logit_maxes = logits.max(1, keepdim=True).values
logits.shape, logit_maxes.shape

2 3 9 5 2    2,9
1 9 1 1 1.   1,9

(torch.Size([32, 27]), torch.Size([32, 1]))

In [ ]:
# h11 h12   w11 w12 w13 w14   l11 l12 . . = h11w11 + h12w21   h11w12+h12w22
# h21 h22.  w21.w22 w22 w24   l21 l22 . . = h21w11 + h22w21.  h21w12+h22w22
# h31 h32                     l31 l32 . . = h31w11 + h32w21.  h31w12+h31w22

# 3x2 2x4   3x4

In [ ]:
# torch.transpose(W2.sum(1, keepdims=True),0 , 1).broadcast_to(batch_size, -1).shape

In [ ]:
bnraw.shape, bndiff.shape, bnvar_inv.shape

(torch.Size([32, 200]), torch.Size([32, 200]), torch.Size([1, 200]))

In [ ]:
bndiff2.shape, bnvar.shape, bnraw.shape

(torch.Size([32, 200]), torch.Size([1, 200]), torch.Size([32, 200]))

In [ ]:
logits.max(1).indices
dlogit_maxes.shape
#dlogits[range(logits.shape[0]), logits.max(1).indices] +=
dlogit_maxes[:,0]

#emb = C[X_batch]
#embcat = emb.reshape(emb.shape[0], -1)
#hprebn = embcat @ W1 + b1

dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

embcat.shape, emb.shape

demb = dembcat.reshape(emb.shape)
emb.shape, C.shape, X_batch.shape

(torch.Size([32, 3, 10]), torch.Size([27, 10]), torch.Size([32, 3]))

In [ ]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)

dlogprobs = torch.zeros((logprobs.shape))
dlogprobs[range(batch_size), Y_batch] = -1/batch_size

dprobs = (1.0/probs) * dlogprobs

dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True)
dcounts = counts_sum_inv * dprobs

dcounts_sum = (-1*counts_sum**-2) * dcounts_sum_inv

dcounts += torch.ones_like(counts) * dcounts_sum

dnorm_logits = counts * dcounts

dlogit_maxes = (-1 * dnorm_logits).sum(1, keepdims=True)

dlogits = dnorm_logits.clone()
dlogits[range(logits.shape[0]), logits.max(1).indices] += dlogit_maxes[:,0]

dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)

dhpreact = (1.0 - h**2) * dh
dbngain = (bnraw * dhpreact).sum(0, keepdims=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdims=True)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdims=True)
dbndiff = bnvar_inv * dbnraw
dbnvar = -0.5*(bnvar+ 1e-5)**-1.5 * dbnvar_inv
dbndiff2 = torch.ones_like(bndiff2) * 1/(batch_size-1) * dbnvar
dbndiff += (2.0*bndiff) * dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-1.0*dbndiff).sum(0)
dhprebn += 1.0/batch_size * torch.ones_like(hprebn) * dbnmeani

dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

demb = dembcat.reshape(emb.shape)

dC = torch.zeros_like(C)
for k in range(X_batch.shape[0]):
  for j in range(X_batch.shape[1]):
    ix = X_batch[k, j]
    dC[ix] += demb[k, j]
'''
 bnmeani = 1/batch_size * hprebn.sum(0, keepdims=True)
  bndiff = hprebn - bnmeani
  bndiff2 = bndiff**2
  bnvar = 1/(batch_size-1) * bndiff2.sum(0, keepdims=True)
  bnvar_inv = (bnvar + 1e-5)**-0.5
  bnraw = bndiff * bnvar_inv
'''
# -----------------

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
bngain          | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
bnbias          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnraw   

## refer for more
https://colab.research.google.com/drive/1WV2oi2fh9XXyldh02wupFQX0wh5ZC-z-?usp=sharing